In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
prev = pd.read_excel('data/DiabetesPrevalence_99.xlsx', sheet_name='prevalence_99', header=1)

In [5]:
prev.head()

,fips,state,year,sex,sex_cat,age,age_cat,prevalence,ll,ul,count
0,1,ALABAMA,2013,0,Overall,1,18-44,0.050086,0.036471,0.063700,85.499
1,2,ALASKA,2013,0,Overall,1,18-44,0.018118,0.009594,0.026643,5.059
2,4,ARIZONA,2013,0,Overall,1,18-44,0.027343,0.013899,0.040787,65.269
3,5,ARKANSAS,2013,0,Overall,1,18-44,0.041642,0.027736,0.055548,43.107
4,6,CALIFORNIA,2013,0,Overall,1,18-44,0.029161,0.022332,0.035989,427.085


In [9]:
# create a separate df for age adjusted
age_adj_prev = prev[prev['age_cat'] == 'age adjusted']

In [10]:
# drop rows where age_cat = age adjusted, since there's no data for ll, ul, count variables
prev = prev[prev['age_cat'] != 'age adjusted']

In [12]:
# drop columns where there is no data available for age adjusted df
age_adj_prev = age_adj_prev.drop(columns = ['ll', 'ul', 'count'], axis=1)
age_adj_prev.head()

,fips,state,year,sex,sex_cat,age,age_cat,prevalence
780,1,ALABAMA,2013,1,Males,9,age adjusted,0.128879
781,2,ALASKA,2013,1,Males,9,age adjusted,0.078849
782,4,ARIZONA,2013,1,Males,9,age adjusted,0.107250
783,5,ARKANSAS,2013,1,Males,9,age adjusted,0.118038
784,6,CALIFORNIA,2013,1,Males,9,age adjusted,0.102623


In [24]:
age_adj_prev[age_adj_prev['state'] == 'United States']

,fips,state,year,sex,sex_cat,age,age_cat,prevalence
831,99,United States,2013,1,Males,9,age adjusted,0.099220
883,99,United States,2013,2,Females,9,age adjusted,0.090170
935,99,United States,2013,0,Overall,9,age adjusted,0.094181
